In [1]:
from transformers import RobertaConfig,RobertaForSequenceClassification,RobertaTokenizer
config = RobertaConfig.from_pretrained("roberta-large")
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

/home/wuyaozong/.conda/envs/fwdgrad_py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /roberta-large/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f58704bf370>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/roberta-large/resolve/main/config.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /roberta-large/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f587045e2b0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/roberta-lar

In [7]:
from transformers import (
    BertConfig,
    BertTokenizer,
    BertForTokenClassification,
)
model = BertForTokenClassification.from_pretrained("bert-base-uncased")

KeyboardInterrupt: 

In [7]:
for i,(key,v) in enumerate(model.named_parameters()):
    print(i,key,v.shape)

0 bert.embeddings.word_embeddings.weight torch.Size([30522, 768])
1 bert.embeddings.position_embeddings.weight torch.Size([512, 768])
2 bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
3 bert.embeddings.LayerNorm.weight torch.Size([768])
4 bert.embeddings.LayerNorm.bias torch.Size([768])
5 bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
6 bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
7 bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
8 bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
9 bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
10 bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
11 bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
12 bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
13 bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
14 bert.encoder.layer.0.attention.output.LayerNorm.

In [11]:
import torch
import copy
global_param = []
fwdgrad = []
for n,p in model.named_parameters():
    if p.requires_grad:
        global_param.append(copy.deepcopy(p.data))
        fwdgrad.append(torch.zeros_like(p.data))
    else:
        global_param.append(None)
        fwdgrad.append(None)

In [13]:
global_param

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 tensor([[ 0.0182,  0.0248,  0.0295,  ...,  0.0189, -0.0234, -0.0112],
         [-0.0293, -0.0280,  0.0166,  ...,  0.0181,  0.0297,  0.0238],
         [-0.0150, -0.0010, -0.0286,  ..., -0.0177, -0.0116, -0.0066],
         ...,
         [ 0.0079, -0.0194, -0.0242,  ..., -0.0303,  0.0280, -0.0023],
         [ 0.0189,  0.0024, -0.0041,  ...,  0.0037,  0.0198,  0.0248],
         [-0.0219,  0.0191, -0.0116,  ...,  0.0160, -0.0100,  0.0010]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 None,
 None,
 None,
 None,
 tensor([[-0.0180,  0.0120,  0.0275,  ..., -0.0276,  0.0023,  0.0240],
         [ 0.0186,  0.0066,  0.0287,  ...,  0.0205,  0.0011, -0.0186],
         [ 0.0301, -0.0182,  0.0303,  ...,  0.0094, -0.0128,  0.0226],
      

In [16]:
for i,(n,p) in enumerate(model.named_parameters()):
    if i == 7:
        print(p)
        print(global_param[i])
        p.data = copy.deepcopy(global_param[i])
        p.data += torch.randn_like(p)

Parameter containing:
tensor([[-0.9383, -0.5986, -1.1644,  ..., -0.5198,  0.1943,  0.4278],
        [ 0.3410,  1.1064, -0.4748,  ...,  1.2348, -0.0469,  0.0342],
        [-1.5556, -1.1835, -0.2540,  ..., -0.0168,  0.4010,  0.0910],
        ...,
        [ 0.0646,  0.5779, -0.2399,  ..., -1.1274, -0.3474,  1.4171],
        [ 0.2972,  1.5144,  0.1560,  ..., -1.6011, -0.3100,  0.7051],
        [ 0.3207, -0.9351, -0.5137,  ...,  1.6552, -0.7981, -1.0951]],
       requires_grad=True)
tensor([[ 0.0182,  0.0248,  0.0295,  ...,  0.0189, -0.0234, -0.0112],
        [-0.0293, -0.0280,  0.0166,  ...,  0.0181,  0.0297,  0.0238],
        [-0.0150, -0.0010, -0.0286,  ..., -0.0177, -0.0116, -0.0066],
        ...,
        [ 0.0079, -0.0194, -0.0242,  ..., -0.0303,  0.0280, -0.0023],
        [ 0.0189,  0.0024, -0.0041,  ...,  0.0037,  0.0198,  0.0248],
        [-0.0219,  0.0191, -0.0116,  ...,  0.0160, -0.0100,  0.0010]])


In [12]:
from transformers import AlbertConfig,AlbertForSequenceClassification,AlbertTokenizer
config = AlbertConfig.from_pretrained("albert-base-v2")
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [1]:
from transformers import DebertaConfig,DebertaForSequenceClassification,DebertaTokenizer
config = DebertaConfig.from_pretrained("microsoft/deberta-xlarge")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-xlarge",ignore_mismatched_sizes=True)
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-xlarge")

/home/wuyaozong/.conda/envs/fwdgrad_py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (

In [13]:
adapter_config = {'original_ln_before':True, 'original_ln_after':True, 'residual_before_ln':True, 'adapter_residual_before_ln':False, 'ln_before':False, 'ln_after':False,
                          'mh_adapter':False, 'output_adapter':True, 'non_linearity':'relu', 'reduction_factor': 64, 'inv_adapter':None, 'inv_adapter_reduction_factor':None,
                          'cross_adapter':False, 'leave_out':[]}
model.add_adapter("aaa",config=adapter_config)

In [14]:
model.train_adapter("aaa")

In [15]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

20750

In [10]:
model

AlbertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (albert): AlbertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(
                  in_features=768, out_features=768, bias=True
          

In [7]:
for n,p in model.named_parameters():
    if "bias" not in n and "classifier" not in n:
        p.requires_grad = False

In [10]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

11650

In [9]:
from torch import nn
model.add_module('pre_classifier',nn.Sequential())

In [11]:
model

AlbertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (albert): AlbertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(
                  in_features=768, out_features=768, bias=True
          

In [2]:
from transformers import DistilBertConfig,DistilBertForSequenceClassification,DistilBertTokenizer
config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [5]:
from transformers.adapters import LoRAConfig
config = LoRAConfig(r=8, alpha=16)
model.add_adapter("lora_adapter", config=config)
model.train_adapter("lora_adapter")

In [6]:
model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict(
                (lora_adapter): LoRA()
              )
            )
            (k_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): Linear(


In [1]:
from transformers import BertConfig,BertForSequenceClassification,BertTokenizer
config = BertConfig.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/home/wuyaozong/.conda/envs/fwdgrad_py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClas

In [12]:
sum(p.numel() for n,p in model.named_parameters())

108893186

In [11]:
import torch
model.bert.add_module('pooler',torch.nn.Sequential())

In [10]:
109483778-768*768-768

108893186

In [13]:
from transformers import BertConfig,BertForSequenceClassification,BertTokenizer
config = BertConfig.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [14]:
model

BertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_fe

In [3]:
import torch

In [4]:
qmodel = torch.quantization.quantize_dynamic(model)

In [5]:
qmodel

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.

In [1]:
import torch.nn as nn
from transformers.adapters.lora import Linear

def convert_lora_to_torch(lora_linear):
    nn_linear = nn.Linear(lora_linear.in_features, lora_linear.out_features)
    nn_linear.weight = lora_linear.weight
    nn_linear.bias = lora_linear.bias
    return nn_linear

def convert_bert_lora_to_torch(bert_model):
    for name, module in bert_model.named_modules():
        if isinstance(module, Linear):
            setattr(module, name, convert_lora_to_torch(module))

convert_bert_lora_to_torch(model)

/home/wuyaozong/.conda/envs/fwdgrad/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [6]:
model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
            (prefix_tuning): PrefixTuningShim(
        

In [9]:
model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
            (prefix_tuning): PrefixTuningShim(
        

In [18]:
index = 0
for name, module in model.named_parameters():
    # print(name,type(module))
    # if name == "bert.encoder.layer.0.attention.self.key":
    print(index,name,module.size(),module.requires_grad)
    index += 1
        # setattr(model, name, convert_lora_to_torch(module))
        

0 albert.embeddings.word_embeddings.weight torch.Size([30000, 128]) False
1 albert.embeddings.position_embeddings.weight torch.Size([512, 128]) False
2 albert.embeddings.token_type_embeddings.weight torch.Size([2, 128]) False
3 albert.embeddings.LayerNorm.weight torch.Size([128]) False
4 albert.embeddings.LayerNorm.bias torch.Size([128]) True
5 albert.encoder.embedding_hidden_mapping_in.weight torch.Size([768, 128]) False
6 albert.encoder.embedding_hidden_mapping_in.bias torch.Size([768]) True
7 albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight torch.Size([768]) False
8 albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias torch.Size([768]) True
9 albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight torch.Size([768, 768]) False
10 albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias torch.Size([768]) True
11 albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight torc

generator

In [8]:
quantized_model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.

In [10]:
quantized_model = torch.quantization.quantize_dynamic(
    model, dtype=torch.qint8
)

In [11]:
quantized_model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.

In [9]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 267.858035
Size (MB): 138.711863


In [84]:
quantized_model

Linear(
  in_features=768, out_features=768, bias=True
  (loras): ModuleDict()
  (bert.encoder.layer.0.attention.self.query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)

In [29]:
def getModelSize(model):
    param_size = 0
    param_sum = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        param_sum += param.nelement()
    buffer_size = 0
    buffer_sum = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
        buffer_sum += buffer.nelement()
    all_size = (param_size + buffer_size) / 1024 / 1024
    print('模型总大小为：{:.3f}MB'.format(all_size))
    return (param_size, param_sum, buffer_size, buffer_sum, all_size)
getModelSize(model)

模型总大小为：255.417MB


(267820040, 66955010, 4096, 512, 255.41699981689453)

In [12]:
import torch
print(model(torch.ones((4,256),dtype=torch.long)))
quantized_model(torch.ones((4,256),dtype=torch.long))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0532,  0.0485],
        [-0.0532,  0.0485],
        [-0.0532,  0.0485],
        [-0.0532,  0.0485]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


SequenceClassifierOutput(loss=None, logits=tensor([[-0.0138,  0.0485],
        [-0.0138,  0.0485],
        [-0.0138,  0.0485],
        [-0.0138,  0.0485]]), hidden_states=None, attentions=None)

In [12]:
torch.quantization.quantize_dynamic(model)

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): Linear(
              in_features=768, out_features=768,

In [9]:
model.add_adapter("aaa")
model.train_adapter("aaa")

In [2]:
import functorch as fc
fmodel,param,buffer = fc.make_functional_with_buffers(model)
param

/home/wuyaozong/.conda/envs/fwdgrad_py38/lib/python3.8/site-packages/torch/_functorch/deprecated.py:101: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.make_functional_with_buffers is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.functional_call instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('make_functional_with_buffers', 'torch.func.functional_call')


(Parameter containing:
 tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
         [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
         [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
         ...,
         [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
         [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
         [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]],
        requires_grad=True),
 Parameter containing:
 tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
           6.8312e-04,  1.5441e-02],
         [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
           2.9753e-02, -5.3247e-03],
         [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
           1.8741e-02, -7.3140e-03],
         ...,
         [ 1.7418e-02,  3.4903e-03, -9.5621e-03,  ...,  2.9599e-03,
           4.3435e-04, -2.6949e-02],
         [ 2.1687e-02, -6.0216e-03,  1.4736e-02,  

In [4]:
len(model.state_dict())

104

In [57]:
for i,p in enumerate(model.named_parameters()):
    print(i,p[0],p[1].requires_grad)

0 distilbert.embeddings.word_embeddings.weight False
1 distilbert.embeddings.position_embeddings.weight False
2 distilbert.embeddings.LayerNorm.weight False
3 distilbert.embeddings.LayerNorm.bias False
4 distilbert.transformer.layer.0.attention.q_lin.weight False
5 distilbert.transformer.layer.0.attention.q_lin.bias False
6 distilbert.transformer.layer.0.attention.k_lin.weight False
7 distilbert.transformer.layer.0.attention.k_lin.bias False
8 distilbert.transformer.layer.0.attention.v_lin.weight False
9 distilbert.transformer.layer.0.attention.v_lin.bias False
10 distilbert.transformer.layer.0.attention.out_lin.weight False
11 distilbert.transformer.layer.0.attention.out_lin.bias False
12 distilbert.transformer.layer.0.sa_layer_norm.weight False
13 distilbert.transformer.layer.0.sa_layer_norm.bias False
14 distilbert.transformer.layer.0.ffn.lin1.weight False
15 distilbert.transformer.layer.0.ffn.lin1.bias False
16 distilbert.transformer.layer.0.ffn.lin2.weight False
17 distilbert.tran

In [52]:
len(param) == len(list(model.named_parameters()))

True

In [53]:
model

DistilBertForSequenceClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): Linear(
              in_features=768, out_features=768,

In [55]:
from torch import nn
model.add_module("pre_classifier",nn.Sequential())

In [31]:
from torch import nn
model.classifier = nn.Linear(1024,4)

In [10]:
for n,p in model.named_parameters():
    if not("bias" in n or "classifier" in n):
        p.requires_grad = False

In [11]:
sum(p.numel() for n,p in model.named_parameters() if p.requires_grad and "bias" in n)

272386

In [34]:
import torch
output = model(torch.ones((8,256),dtype=torch.int64))

odict_keys(['logits'])

In [37]:
output[0]

tensor([[[-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         ...,
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884]],

        [[-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         ...,
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884]],

        [[-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         ...,
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884]],

        ...,

        [[-0.3631,  0.1472, -1.0191,  0.3884],
         [-0.3631,  0.1472, -1.0191,  0.3884]

In [5]:
from transformers import AlbertTokenizer, AlbertModel
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained("albert-base-v2")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model.add_adapter

AttributeError: 'AlbertModel' object has no attribute 'add_adapter'